In [1]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd
from faker import Faker
fake = Faker()

In [7]:

engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT code, AnswerTable FROM 
fsma_QuestionTypes
WHERE code LIKE 'NCEinstein%' AND code NOT LIKE '%ARTIFACT%'
"""  
df = pd.read_sql(query,con=conn)
df.set_index('code', inplace=True )
conn.close()
df

,AnswerTable
code,
NCEinsteinPublicIP,EinsteinPublicIP
NCEinsteinUnannounced,EinsteinUnannounced
NCEinsteinBGP,EinsteinBGP
NCEinsteinUnmonitoredTraffic,EinsteinUnmonitoredTraffic
NCEinsteinTaps,EinsteinTaps
NCEinsteinNATInfo,EinsteinNATInfo
NCEinsteinNetworkChannels,EinsteinNetworkChannels
NCEinsteinVLAN,EinsteinVLAN


In [46]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, IdText, QuestionTypeCode, QuestionTypeID, QTEXT, help_text   
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%EINS%'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()
for i,r in df.iterrows():
    s = ''
    s = s + f" -- {str(r['QPK'])}  {str(r['QGROUP'])}"
    print( s ) 
for i,r in df.iterrows():
    qtext = r['QTEXT'].replace("'","''").replace("\n","").replace("\r","").replace("\t","")
    help_text = r['help_text'].replace("'","''").replace("\n","").replace("\r","").replace("\t","")
    s = f"UPDATE fsma_Questions \nSET QuestionText = '{qtext}'" 
    s = s + f"\n, help_text = '{help_text}'"
    s = s + f'\n, FK_QuestionType = ' + str(r['QuestionTypeID']) + ' ' 
    s = s + f"WHERE PK_Question = " + str(r['QPK']) + f";\n"
    s = s + f" -- OLD QGROUP {str(r['QGROUP'])} TYPE {str(r['QuestionTypeCode'])}\n"
    print( s ) 

 -- 20850  2110
 -- 20861  2111
 -- 20881  2113
 -- 20891  2114
 -- 20900  2115
 -- 20940  2120
 -- 20960  2122
 -- 20950  2121
UPDATE fsma_Questions 
SET QuestionText = '<p>EINSTEIN requires a complete specification of what IP addresses are associated with each agency''s network. This allows CISA to  identify the target of suspected cyber activity. </p><p>Please complete the table below and supply all IP addresses, defined as globally routable unicast IP addresses, that are assigned to your network.  Include both announced and unannounced IP addresses. It is not necessary to include the EINSTEIN management IP addresses that CISA uses to administer the EINSTEIN servers.</p>'
, help_text = '<h4>Data Entry Tips:</h4><p><strong>ASN or IP Address Block: </strong>Public Autonomous System Number (ASN): (e.g., "220") or Address or Classless Inter-Domain Routing (CIDR): Examples: 123.123.123.0/24, 67.78.56.12/28, 02::1234:abcd:ffff:c0a8:101/64. NOTE: In lieu of CIDR, address ranges (starting a

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, IdText, QuestionTypeCode , QuestionTypeCode, QTEXT, help_text   
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%EINS%'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()

In [ ]:
sections = [ 
{'1': 'Public IP Address Specification'}
{'2': 'Unannounced IP Address Specification'}
{'3': 'EINSTEIN Test Access Point (TAP) Locations'}
{'4': 'Virtual LANs (VLANs) Specification'}
{'5': 'Unmonitored Internet Traffic'}
{'6': 'Network Address Translation (NAT) Specification'}
{'7': 'Border Gateway Protocol (BGP) Specification'}
{'8': 'Network Diagrams'}        
]


In [11]:
def getip(): 
    return f"{np.random.randint(1, 255)}.{np.random.randint(1, 255)}.{np.random.randint(1, 255)}.{np.random.randint(1, 255)}"
getip()

'70.233.135.81'

In [16]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT TOP 1 * FROM EinsteinPublicIP 
"""  
df = pd.read_sql(query,con=conn)  
df.drop('PK_EinsteinPublicIP', axis=1, inplace=True)
df.drop('PK_OrgSubmission', axis=1, inplace=True)
df.drop('UserId', axis=1, inplace=True)
df.drop('isActive', axis=1, inplace=True)
df.drop('DateDeactivated', axis=1, inplace=True)
for c in df.columns:
    print(f"d['{c}'] = '{getip()}'"  )

 

d['Name'] = getip()
d['ISP'] = getip()
d['OtherISP'] = getip()
d['ExternalOrg'] = getip()
d['Visible'] = getip()
d['StartingIP'] = getip()
d['EndingIP'] = getip()
d['TIC_MTIPS'] = getip()


In [36]:
lst = []
ln = 20000
for i in range(ln):
    d={}
    d['StartingIP'] = getip()
    d['EndingIP'] = getip()    
    d['Name_Purpose'] = getip()
    d['ISP'] = 'IBM' 
    d['OtherISP'] = 'Other'
    d['External_Organization'] = fake.address()[:25]
    d['TIC_MTIPS'] = 'TIC'
    d['Visible'] = 'Yes' 
    lst.append(d)
df=pd.DataFrame(lst)
df.to_excel(fr'c:\temp\EinsteinPublicIP_{ln}.xlsx', index=False)    
